In [56]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D,ZeroPadding1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv1D,UpSampling1D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

In [95]:
#print(tf.__version__)

2.0.0-alpha0


In [ ]:
#!pip3 install gast==0.2.2

In [5]:
# Size vector to generate images from
SEED_SIZE = 100

# Configuration
BATCH_SIZE = 32
BUFFER_SIZE = 60000


Will generate 96px square images.


In [4]:
!ls

Music Generator.ipynb     faces                     midi-lstm-gan
Pokemon code tryout.ipynb image GANs.ipynb
face_images               maestro-v1.0.0


# Midi data

In [58]:
training_data_midi = np.load('All_Maestro_Parsed.npy')
#training_data_midi=np.load('All_pianoroll_v1.npy')

In [59]:
training_data_midi.shape

(15300, 200, 3)

In [60]:
train_data_midi_temp = training_data_midi[:3000,:192,:]

In [61]:
train_data_midi_temp.shape

(3000, 192, 3)

In [62]:
type(train_data_midi_temp)

numpy.ndarray

In [63]:
train_data_midi_temp

array([[[1.81818182e-01, 8.45376135e-03, 0.00000000e+00],
        [2.61363636e-01, 9.27628948e-02, 1.71360027e-04],
        [1.70454545e-01, 1.23379220e-02, 7.19712115e-03],
        ...,
        [1.81818182e-01, 1.59936026e-03, 1.42800023e-03],
        [2.61363636e-01, 2.74176044e-03, 8.56800137e-04],
        [4.54545455e-01, 1.14240018e-03, 3.99840064e-04]],

       [[1.93181818e-01, 8.45376135e-03, 0.00000000e+00],
        [2.72727273e-01, 9.27628948e-02, 1.71360027e-04],
        [1.81818182e-01, 1.23379220e-02, 7.19712115e-03],
        ...,
        [1.93181818e-01, 1.59936026e-03, 1.42800023e-03],
        [2.72727273e-01, 2.74176044e-03, 8.56800137e-04],
        [4.65909091e-01, 1.14240018e-03, 3.99840064e-04]],

       [[2.04545455e-01, 8.45376135e-03, 0.00000000e+00],
        [2.84090909e-01, 9.27628948e-02, 1.71360027e-04],
        [1.93181818e-01, 1.23379220e-02, 7.19712115e-03],
        ...,
        [2.04545455e-01, 1.59936026e-03, 1.42800023e-03],
        [2.84090909e-01, 2.74

In [64]:
train_data_midi_temp =np.float32(train_data_midi_temp)

In [66]:
training_data_midi[0].shape

(200, 3)

In [67]:
train_dataset_midi = tf.data.Dataset.from_tensor_slices(train_data_midi_temp) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [68]:
train_dataset_midi

<BatchDataset shapes: (None, 192, 3), types: tf.float32>

In [69]:
training_data_midi[0]

array([[1.81818182e-01, 8.45376135e-03, 0.00000000e+00],
       [2.61363636e-01, 9.27628948e-02, 1.71360027e-04],
       [1.70454545e-01, 1.23379220e-02, 7.19712115e-03],
       [1.25000000e-01, 5.94048095e-02, 7.19712115e-03],
       [1.81818182e-01, 6.07756897e-02, 5.27217684e-02],
       [2.72727273e-01, 2.09630434e-02, 1.73073628e-02],
       [2.50000000e-01, 2.23339236e-02, 1.57651225e-02],
       [2.61363636e-01, 9.82464157e-02, 1.97635232e-02],
       [1.70454545e-01, 7.59696122e-03, 7.31136117e-03],
       [1.25000000e-01, 5.85480094e-02, 8.22528132e-03],
       [1.81818182e-01, 6.32318501e-02, 5.12937682e-02],
       [2.72727273e-01, 2.51328040e-02, 2.16484835e-02],
       [2.50000000e-01, 1.81641629e-02, 1.79356829e-02],
       [2.61363636e-01, 2.62809162e-01, 1.58793625e-02],
       [1.70454545e-01, 1.02244816e-02, 7.31136117e-03],
       [1.25000000e-01, 3.90129662e-02, 8.79648141e-03],
       [1.81818182e-01, 2.05632033e-03, 4.03838465e-02],
       [1.81818182e-01, 4.16976

In [70]:
training_data_midi[:,:,1] = training_data_midi[:,:,1]*100
training_data_midi[:,:,2] = training_data_midi[:,:,2]*100

In [77]:
def build_generator(seed_size, channels):
    model = Sequential()

    model.add(Dense(4*256,activation="relu",input_dim=SEED_SIZE))
    model.add(Reshape((4,256)))

    model.add(UpSampling1D()) #8
    model.add(Conv1D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling1D()) #16
    model.add(Conv1D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    # Output resolution, additional upsampling
    model.add(UpSampling1D()) #32
    model.add(Conv1D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    
    model.add(UpSampling1D()) #64
    model.add(Conv1D(64,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling1D(size=(3))) #
    model.add(Conv1D(32,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv1D(3,kernel_size=3,padding="same"))
    model.add(Activation("sigmoid"))
    
    return model

    
    
def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv1D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding1D(padding=((0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv1D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [78]:
generator = build_generator(SEED_SIZE, 1)

noise = tf.random.normal([1,SEED_SIZE])
generated_image = generator(noise, training=False)

#plt.imshow(generated_image[0, :, :, 0])

In [79]:
type(generator)

tensorflow.python.keras.engine.sequential.Sequential

In [80]:
image_shape = (192,3)

discriminator = build_discriminator(image_shape)
decision = discriminator(generated_image)
print (decision)

tf.Tensor([[0.49777135]], shape=(1, 1), dtype=float32)


In [81]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [82]:
generator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)

In [83]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(seed, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    

        gradients_of_generator = gen_tape.gradient(\
            gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(\
            disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(
            gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(
            gradients_of_discriminator, 
            discriminator.trainable_variables))
    return gen_loss,disc_loss

In [85]:
def train(dataset, epochs):
    #fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 
    #                                   SEED_SIZE))
    start = time.time()

    for epoch in range(epochs):
        epoch_start = time.time()

        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            t = train_step(image_batch)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])

        g_loss = sum(gen_loss_list) / len(gen_loss_list)
        d_loss = sum(disc_loss_list) / len(disc_loss_list)

        epoch_elapsed = time.time()-epoch_start
        print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss},'\
           ' {hms_string(epoch_elapsed)}')
        #save_images(epoch,fixed_seed)

    elapsed = time.time()-start
    print (f'Training time: {hms_string(elapsed)}')

In [15]:
#!pip3 install gast==0.2.2

In [86]:
train(train_dataset_midi, 50)

Epoch 1, gen loss=2.702883720397949,disc loss=0.507816731929779, {hms_string(epoch_elapsed)}
Epoch 2, gen loss=5.312332630157471,disc loss=0.04068141430616379, {hms_string(epoch_elapsed)}
Epoch 3, gen loss=5.952878952026367,disc loss=0.0418538972735405, {hms_string(epoch_elapsed)}
Epoch 4, gen loss=6.1130051612854,disc loss=0.08918022364377975, {hms_string(epoch_elapsed)}
Epoch 5, gen loss=3.7477307319641113,disc loss=0.40557876229286194, {hms_string(epoch_elapsed)}
Epoch 6, gen loss=2.439633369445801,disc loss=1.2030308246612549, {hms_string(epoch_elapsed)}
Epoch 7, gen loss=1.6988714933395386,disc loss=1.38587486743927, {hms_string(epoch_elapsed)}
Epoch 8, gen loss=1.5718005895614624,disc loss=1.466781735420227, {hms_string(epoch_elapsed)}
Epoch 9, gen loss=1.3363851308822632,disc loss=1.424028754234314, {hms_string(epoch_elapsed)}
Epoch 10, gen loss=1.3053491115570068,disc loss=1.4275621175765991, {hms_string(epoch_elapsed)}
Epoch 11, gen loss=1.3033792972564697,disc loss=1.34341835

In [17]:
generator.save(os.path.join(DATA_PATH,"face_generator.h5"))

In [100]:
noise = tf.random.normal([1, SEED_SIZE])
generated_image = generator(noise, training=False)

#plt.imshow(generated_image[0, :, :, 0])

In [101]:
generated_image.numpy()


array([[[6.12866163e-01, 5.05388379e-02, 4.88343835e-03],
        [4.25765544e-01, 9.80985165e-03, 6.85393810e-04],
        [5.57674766e-01, 5.93489408e-03, 2.58803368e-03],
        [3.46866488e-01, 1.98981166e-03, 2.57781148e-03],
        [2.50857115e-01, 1.49756670e-03, 4.69464064e-03],
        [4.10543084e-01, 1.29908323e-03, 2.44653225e-03],
        [4.60865200e-01, 1.26892328e-03, 2.90474296e-03],
        [4.34806108e-01, 1.46114826e-03, 1.81552768e-03],
        [7.05136120e-01, 9.80705023e-04, 2.06518173e-03],
        [5.94105780e-01, 1.46114826e-03, 3.72406840e-03],
        [7.45148063e-01, 1.53812766e-03, 2.75519490e-03],
        [5.83535910e-01, 2.39512324e-03, 2.37861276e-03],
        [5.09982765e-01, 1.95956230e-03, 1.96644664e-03],
        [6.90756619e-01, 2.14254856e-03, 7.38978386e-04],
        [7.05254257e-01, 1.20320916e-03, 3.91393900e-04],
        [6.69434071e-01, 1.26147270e-03, 3.45528126e-04],
        [7.35527515e-01, 2.73147225e-03, 3.84747982e-04],
        [6.198

In [308]:
generated_image.numpy()[0].shape

(512, 128, 1)

In [230]:
generated_image[0]

<tf.Tensor: id=584142, shape=(192, 3), dtype=float32, numpy=
array([[5.53601801e-01, 2.35945582e-02, 4.72706556e-03],
       [6.00084066e-01, 1.31965578e-02, 1.30796432e-03],
       [5.97620964e-01, 1.09574795e-02, 1.84834003e-03],
       [6.08771026e-01, 1.31816268e-02, 2.37160921e-03],
       [5.47196269e-01, 1.72084570e-02, 2.77766585e-03],
       [5.84240794e-01, 2.02982426e-02, 5.81037998e-03],
       [5.42351544e-01, 1.95013583e-02, 6.30962849e-03],
       [4.26500350e-01, 1.14898980e-02, 3.94293666e-03],
       [5.63423872e-01, 1.13964975e-02, 2.85965204e-03],
       [5.55652142e-01, 1.08247399e-02, 2.26536393e-03],
       [5.39073348e-01, 9.71010327e-03, 2.50259042e-03],
       [5.31926632e-01, 1.08327270e-02, 2.83476710e-03],
       [5.54143429e-01, 1.02418065e-02, 2.83679366e-03],
       [5.19265175e-01, 9.94595885e-03, 3.85132432e-03],
       [5.87469578e-01, 9.37476754e-03, 3.44538689e-03],
       [5.77217638e-01, 5.47808409e-03, 3.30707431e-03],
       [5.40466309e-01, 3.7

In [102]:
from mido import MidiFile, MidiTrack, Message
from music21 import *
from IPython.display import Image
import pandas as pd
import numpy as np
import math
import glob

In [34]:
def recreate_midi(df_first_notes, speed=20000):
    # function to take a dataframe created by something like parse_notes() or a gan and return a midi
    
    # Can start by reverse scaling the note:
    df_reversed = df_first_notes.copy()
    df_reversed['note'] = round(df_reversed['note'] * 88 + 20)  # might want to have something more special than round()
    df_reversed.note = df_reversed.note.astype(int)
    df_reversed['velocity'] = 60  # create a uniform middling velocity

    # recreate the absolute time index and drop time_since_last (we'll recreate it with the stop signals)
    df_reversed['time_index'] = df_reversed.time_since_last.cumsum()
    df_reversed = df_reversed.drop(columns = 'time_since_last')

    # create a stop signal for each note at the appropriate time_index:
    for i in range(len(df_reversed)):
        stop_note = pd.DataFrame([[df_reversed.note[i], 0, 0, df_reversed.duration[i] + df_reversed.time_index[i]]],
                                 columns=['note', 'duration', 'velocity', 'time_index'])
        df_reversed = df_reversed.append(stop_note, ignore_index=True)
    df_reversed = df_reversed.sort_values('time_index').reset_index(drop=True)

    # recreate time_since last with the stop note signals
    df_reversed['time'] = [0] + [df_reversed.time_index[i+1] - df_reversed.time_index[i] 
                                 for i in range(len(df_reversed)-1)]
    # and now we don't need duration or time_index so can drop those
    df_reversed = df_reversed.drop(columns = {'time_index','duration'})

    # finally, we need to scale the time since last note appropriately:
    df_reversed['time'] = round(df_reversed['time'] * speed)
    df_reversed.time = df_reversed.time.astype(int)

    # finally, recreate the midi and return
    mid_remade = MidiFile()
    track = MidiTrack()
    mid_remade.tracks.append(track)
    track.append(Message('program_change', program=0, time=0))
    for i in range(len(df_reversed)):
        track.append(Message('note_on', note=df_reversed.note[i], velocity=df_reversed.velocity[i], time=df_reversed.time[i]))

    return mid_remade

In [29]:
import pandas as pd

In [103]:
midi_dat = generated_image.numpy()
midi_dat[0].shape

midi_df = pd.DataFrame(midi_dat[0],columns=["note", "duration",'time_since_last'])

In [104]:
midi_df.head()

,note,duration,time_since_last
0,0.612866,0.050539,0.004883
1,0.425766,0.009810,0.000685
2,0.557675,0.005935,0.002588
3,0.346866,0.001990,0.002578
4,0.250857,0.001498,0.004695


In [105]:
mid_remade = recreate_midi(midi_df)

In [106]:
mid_remade.save('mid_test_2.mid')

In [107]:
# see if we can play in jupyter notebook
mf = midi.MidiFile()
mf.open('mid_test_2.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')